In [113]:
# hide

%pip install pandas
%pip install numpy
%pip install scikit-learn==1.5.2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [114]:
import pandas as pd #type: ignore
import numpy as np #type: ignore

In [115]:
df_classification = pd.read_csv("./train_taxi.csv")

In [116]:
df_classification.head()

,ID,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,trip_type,congestion_surcharge,payment_type
0,T03315,2.0,2021-07-02 17:19:11,2021-07-02 17:40:02,N,1.0,152,142,1.0,4.30,...,1.0,0.5,5.00,0.00,NaN,0.3,27.55,1.0,2.75,1.0
1,T07720,2.0,2021-07-05 22:48:04,2021-07-05 23:12:02,N,1.0,93,107,2.0,13.52,...,0.5,0.5,12.15,6.55,NaN,0.3,60.75,1.0,2.75,1.0
2,T09695,2.0,2021-07-07 06:29:43,2021-07-07 06:38:42,N,1.0,74,75,1.0,1.55,...,0.0,0.5,0.00,0.00,NaN,0.3,8.80,1.0,0.00,2.0
3,T08802,2.0,2021-07-06 15:33:29,2021-07-06 15:43:25,N,1.0,226,129,1.0,1.22,...,0.0,0.5,0.00,0.00,NaN,0.3,8.80,1.0,0.00,2.0
4,T01413,2.0,2021-07-01 17:09:07,2021-07-01 18:03:12,N,1.0,55,86,1.0,14.23,...,1.0,0.5,0.00,2.45,NaN,0.3,49.75,1.0,0.00,1.0


In [117]:
df_classification = df_classification.drop(columns=['ehail_fee'])

In [118]:
df_classification = df_classification[df_classification['payment_type'].isin([1,2])]

In [119]:
df_classification =df_classification.drop(columns=['ID'])

In [120]:
df_classification = df_classification.drop(columns=['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'passenger_count', 'improvement_surcharge', 'congestion_surcharge'], axis=1)

In [121]:
df_classification['payment_type'] = df_classification['payment_type'].replace({
    2: 0,
    1: 1
})

In [122]:
df_classification['is_give_tip_using_credit_card'] = np.where(df_classification['tip_amount'] > 0, 1, 0)

In [123]:
df_classification['is_busy_or_night'] = np.where(df_classification['extra'] > 0, 1, 0)

In [124]:
trip_one_hot = pd.get_dummies(df_classification['trip_type'], prefix='trip_type').astype(int)

In [125]:
ratecode_id_one_hot = pd.get_dummies(df_classification['RatecodeID'], prefix='ratecode_id').astype(int)

In [126]:
vendor_id_one_hot= pd.get_dummies(df_classification['VendorID'], prefix='vendor_id').astype(int)

In [127]:
store_and_fwd_flag = pd.get_dummies(df_classification['store_and_fwd_flag'], prefix='store_and_fwd_flag').astype(int)

In [128]:
df_classification = pd.concat([df_classification, trip_one_hot, ratecode_id_one_hot, vendor_id_one_hot, store_and_fwd_flag], axis=1)
df_classification = df_classification.drop(columns=['trip_type', 'RatecodeID', 'VendorID', 'store_and_fwd_flag', 'mta_tax', 'extra'], axis=1)

In [129]:
X = df_classification.drop(columns='payment_type', axis=1)
y = df_classification['payment_type']

In [130]:
from sklearn.preprocessing import RobustScaler # type: ignore

scaled_features = ['trip_distance', 'total_amount', 'fare_amount', 'tolls_amount', 'tip_amount']

scaler = RobustScaler()
scaler = scaler.fit(X[scaled_features])
X[scaled_features] = scaler.transform(X[scaled_features])

In [131]:
X

,trip_distance,fare_amount,tip_amount,tolls_amount,total_amount,is_give_tip_using_credit_card,is_busy_or_night,trip_type_1.0,trip_type_2.0,ratecode_id_1.0,ratecode_id_2.0,ratecode_id_3.0,ratecode_id_4.0,ratecode_id_5.0,vendor_id_1.0,vendor_id_2.0,store_and_fwd_flag_N,store_and_fwd_flag_Y
0,0.855072,0.75,2.427184,0.00,1.187500,1,1,1,0,1,0,0,0,0,0,1,1,0
1,4.195652,2.75,5.898058,6.55,3.954167,1,1,1,0,1,0,0,0,0,0,1,1,0
2,-0.141304,-0.25,0.000000,0.00,-0.375000,0,0,1,0,1,0,0,0,0,0,1,1,0
3,-0.260870,-0.25,0.000000,0.00,-0.375000,0,0,1,0,1,0,0,0,0,0,1,1,0
4,4.452899,3.50,0.000000,2.45,3.037500,0,1,1,0,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,-0.195652,-0.40,0.800971,0.00,-0.279167,1,1,1,0,1,0,0,0,0,1,0,1,0
7996,0.927536,0.60,0.000000,0.00,0.562500,0,1,1,0,1,0,0,0,0,1,0,1,0
7997,-0.423913,-0.55,0.000000,0.00,-0.625000,0,0,1,0,1,0,0,0,0,0,1,1,0
7998,2.336957,1.45,4.155340,0.00,1.984167,1,0,1,0,1,0,0,0,0,0,1,1,0


In [132]:
from sklearn.model_selection import train_test_split #type: ignore

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [133]:
from sklearn.linear_model import LogisticRegressionCV #type: ignore

model = LogisticRegressionCV(solver='liblinear',penalty='l1', cv=5, n_jobs=-1, random_state=42, max_iter=10000)

In [134]:
model.fit(X_train, y_train)

LogisticRegressionCV(cv=5, max_iter=10000, n_jobs=-1, penalty='l1',
                     random_state=42, solver='liblinear')

In [135]:
y_pred = model.predict(X_test)

In [136]:
from sklearn.metrics import f1_score #type: ignore


f1 = f1_score(y_test, y_pred, average='macro')

print(f'F1 score: {f1:.4f}')

F1 score: 0.9184


In [137]:
df_test_classification = pd.read_csv("./taxi_trip_classif.csv")

In [138]:
df_test_classification = df_test_classification.drop(columns=['ehail_fee'], axis=1)

In [139]:
processed_df_test_classification = df_test_classification.copy()
processed_df_test_classification = processed_df_test_classification.drop(columns='ID', axis=1)

In [140]:
processed_df_test_classification = processed_df_test_classification.drop(columns=['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'passenger_count', 'improvement_surcharge', 'congestion_surcharge'], axis=1)

In [141]:
processed_df_test_classification['is_give_tip_using_credit_card'] = np.where(processed_df_test_classification['tip_amount'] > 0, 1, 0)

In [142]:
processed_df_test_classification['is_busy_or_night'] = np.where(processed_df_test_classification['extra'] > 0, 1, 0)

In [143]:
trip_one_hot = pd.get_dummies(processed_df_test_classification['trip_type'], prefix='trip_type').astype(int)
ratecode_id_one_hot = pd.get_dummies(processed_df_test_classification['RatecodeID'], prefix='ratecode_id').astype(int)
vendor_id_one_hot= pd.get_dummies(processed_df_test_classification['VendorID'], prefix='vendor_id').astype(int)
store_and_fwd_flag = pd.get_dummies(processed_df_test_classification['store_and_fwd_flag'], prefix='store_and_fwd_flag').astype(int)

In [144]:
processed_df_test_classification = pd.concat([processed_df_test_classification, trip_one_hot, ratecode_id_one_hot, vendor_id_one_hot, store_and_fwd_flag], axis=1)
processed_df_test_classification = processed_df_test_classification.drop(columns=['trip_type', 'RatecodeID', 'VendorID', 'store_and_fwd_flag', 'mta_tax', 'extra'], axis=1)

In [145]:
processed_df_test_classification[scaled_features] = scaler.transform(processed_df_test_classification[scaled_features])

In [146]:
y_result = model.predict(processed_df_test_classification)

In [147]:
csv_result = pd.DataFrame({
    "ID": df_test_classification["ID"],
    "payment_type": y_result
})

csv_result["payment_type"] = csv_result["payment_type"].replace({
    0: 2,
    1: 1
})

In [148]:
csv_result

,ID,payment_type
0,T05226,2.0
1,T02377,1.0
2,T08294,2.0
3,T00815,1.0
4,T04025,1.0
...,...,...
1595,T07378,1.0
1596,T01681,1.0
1597,T03778,2.0
1598,T08415,2.0


In [150]:
from datetime import datetime
from zoneinfo import ZoneInfo

utc_plus_7 = ZoneInfo("Asia/Bangkok")
current_iso_timestamp = datetime.now(utc_plus_7).isoformat()

filename = f"submission_kasdead_klasifikasi_logistic_{current_iso_timestamp}.csv"
csv_result.to_csv(filename, index=False)